In [16]:
import numpy as np
import rasterio
from rasterio.transform import from_bounds

def process_dem_with_river_depth(dem_path, river_depth_path, output_path):
    """
    Processes a DEM by lifting it by the maximum river depth value and then subtracting the river depth map 
    to make the DEM resemble the riverbed. The output is saved as a GeoTIFF.
    """
    # Read the DEM (Digital Elevation Model)
    with rasterio.open(dem_path) as dem_src:
        dem_data = dem_src.read(1)  # Read the first band (the DEM itself)
        dem_meta = dem_src.meta.copy()  # Metadata to copy for the output
        dem_crs = dem_src.crs  # Coordinate Reference System
        dem_transform = dem_src.transform  # Affine transformation for georeferencing

    # Read the River Depth Map
    with rasterio.open(river_depth_path) as river_src:
        river_depth_data = river_src.read(1)  # Read the first band (the river depth)
        river_crs = river_src.crs  # Coordinate Reference System
        river_transform = river_src.transform  # Affine transformation for georeferencing

    # Ensure that the DEM and river depth map have the same CRS and dimensions
    if dem_crs != river_crs:
        raise ValueError(f"CRS mismatch: DEM CRS ({dem_crs}) and River Depth CRS ({river_crs}) must be the same.")
    
    if dem_data.shape != river_depth_data.shape:
        raise ValueError(f"DEM and river depth map must have the same dimensions. \n DEM shape: {dem_data.shape} \n River Depth shape: {river_depth_data.shape}")

    # Step 1: Lift the DEM by the maximum river depth value
    max_river_depth = np.nanmax(river_depth_data)  # Maximum river depth value
    lifted_dem = dem_data + max_river_depth  # Lift the DEM by the max river depth

    # Step 2: Subtract the river depth from the lifted DEM
    riverbed_dem = lifted_dem - river_depth_data  # Subtract the river depth to get the riverbed DEM

    # Step 3: Ensure the result is within valid range (e.g., non-negative)
    riverbed_dem = np.clip(riverbed_dem, 0, None)  # Ensure no negative values (if that's required)

    # Update the metadata to reflect the new dimensions (it should be the same as the input DEM)
    dem_meta.update({
        "dtype": "float32",  # Adjust the data type for more precision in depth values
        "count": 1,  # Only one band (final DEM)
        "crs": dem_crs,  # Retain the original CRS
        "transform": dem_transform  # Retain the original transform
    })

    # Step 4: Save the processed DEM to the output file as GeoTIFF
    with rasterio.open(output_path, 'w', **dem_meta) as dst:
        dst.write(riverbed_dem, 1)

    print(f"Processed DEM saved to {output_path}")


In [8]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def reproject_tiff(input_path, output_path, src_crs="EPSG:2056", dst_crs="EPSG:21781"):
    """
    Reprojects a GeoTIFF from one CRS to another.
    
    :param input_path: Path to the input GeoTIFF file.
    :param output_path: Path to save the reprojected GeoTIFF file.
    :param src_crs: Source CRS in EPSG format (default: "EPSG:2056").
    :param dst_crs: Destination CRS in EPSG format (default: "EPSG:21781").
    """
    # Open the source raster file
    with rasterio.open(input_path) as src:
        # Get the metadata of the source file
        src_meta = src.meta.copy()
        
        # Calculate the transform and dimensions for the target CRS
        transform, width, height = calculate_default_transform(
            src_crs, dst_crs, src.width, src.height, *src.bounds)
        
        # Update the metadata for the output file
        src_meta.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        # Create the output file
        with rasterio.open(output_path, 'w', **src_meta) as dst:
            # Reproject each band
            for i in range(1, src.count + 1):
                # Read the source band
                src_band = src.read(i)
                
                # Prepare an empty array for the destination band
                dst_band = np.empty((height, width), dtype=src_band.dtype)
                
                # Reproject the band into the new CRS
                reproject(
                    src_band,  # Source band
                    dst_band,  # Destination band
                    src_transform=src.transform,  # Source transform
                    src_crs=src.crs,  # Source CRS
                    dst_transform=transform,  # Destination transform
                    dst_crs=dst_crs,  # Destination CRS
                    resampling=Resampling.nearest  # Resampling method
                )
                
                # Write the reprojected band to the destination file
                dst.write(dst_band, i)
    
    print(f"Reprojected file saved to {output_path}")


In [27]:
import rasterio
import numpy as np
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.windows import from_bounds

def subtract_raster_based_on_coordinates(dem_path, river_depth_path, output_path):
    """
    Subtracts a river depth map from a DEM, aligning them by their coordinates.
    If necessary, resamples the river depth map to the DEM grid before subtraction.
    
    :param dem_path: Path to the DEM GeoTIFF.
    :param river_depth_path: Path to the river depth GeoTIFF.
    :param output_path: Path to save the resulting GeoTIFF after subtraction.
    """
    
    # Open the river depth file first to handle the reprojection
    with rasterio.open(river_depth_path) as river_src:
        river_depth_data = river_src.read(1)
        river_crs = river_src.crs
        river_transform = river_src.transform
        river_bounds = river_src.bounds
        
    # Open the DEM file and perform the cropping inside the with block
    with rasterio.open(dem_path) as dem_src:
        dem_data = dem_src.read(1)
        dem_crs = dem_src.crs
        dem_transform = dem_src.transform
        dem_meta = dem_src.meta
        dem_bounds = dem_src.bounds
    
        # Check if CRS and transform match
        if dem_crs != river_crs:
            raise ValueError(f"CRS mismatch: DEM CRS ({dem_crs}) and River Depth CRS ({river_crs}) must be the same.")
        
        # Calculate the target transform and size for alignment
        transform, width, height = calculate_default_transform(
            river_crs, dem_crs, river_depth_data.shape[1], river_depth_data.shape[0], *river_bounds)
        
        # Create a new array for the reprojected river depth data (resampled to DEM grid)
        reprojected_river_depth = np.empty((height, width), dtype=np.float32)

        # Reproject river depth data onto the DEM grid (use nearest-neighbor resampling here)
        reproject(
            river_depth_data,  # Source band
            reprojected_river_depth,  # Destination band
            src_transform=river_transform,
            src_crs=river_crs,
            dst_transform=transform,
            dst_crs=dem_crs,
            resampling=Resampling.nearest
        )
        
        # Crop both rasters to the common bounding box
        common_bounds = (max(dem_bounds[0], river_bounds[0]),  # min x
                         max(dem_bounds[1], river_bounds[1]),  # min y
                         min(dem_bounds[2], river_bounds[2]),  # max x
                         min(dem_bounds[3], river_bounds[3]))  # max y
        
        # Get the window (pixel coordinates) for the common bounds
        window = from_bounds(*common_bounds, transform=dem_transform)
        
        # Crop both DEM and reprojected river depth data to the same window
        dem_cropped = dem_src.read(1, window=window)
        river_depth_cropped = reprojected_river_depth[window.toslices()]  # Corrected usage of tosllices()

        # Perform the subtraction: DEM - River Depth
        riverbed_dem = dem_cropped - river_depth_cropped

        # Update the metadata for the output file (ensure it uses DEM's transform and CRS)
        dem_meta.update({
            'dtype': 'float32',  # Adjust the data type to float32 for precision
            'crs': dem_crs,
            'transform': dem_transform,
            'count': 1  # We are writing a single band (the result of the subtraction)
        })

        # Write the resulting image to a new GeoTIFF file
        with rasterio.open(output_path, 'w', **dem_meta) as dst:
            dst.write(riverbed_dem, 1)

        print(f"Processed DEM saved at: {output_path}")


In [ ]:
dem = "/Volumes/T7 Shield/GMP_Data/processed_data/05_DEM/dem/height_map_old_national_1975_interpolated.tif"
river = "/Volumes/T7 Shield/GMP_Data/processed_data/05_DEM/depth_maps/resampled/river_depth_raster_1975.tif"

reproject_tiff(dem,dem.replace(".tif","_reprojected.tif"))


Reprojected file saved to /Volumes/T7 Shield/GMP_Data/processed_data/05_DEM/dem/height_map_old_national_1975_interpolated_reprojected.tif


In [28]:
dem = "/Volumes/T7 Shield/GMP_Data/processed_data/05_DEM/dem/resampled/height_map_old_national_1975_interpolated_reprojected.tif"
river = "/Volumes/T7 Shield/GMP_Data/processed_data/05_DEM/depth_maps/resampled/river_depth_raster_1975.tif"
subtract_raster_based_on_coordinates(dem,river,dem.replace(".tif","_riverbed.tif"))

Processed DEM saved at: /Volumes/T7 Shield/GMP_Data/processed_data/05_DEM/dem/resampled/height_map_old_national_1975_interpolated_reprojected_riverbed.tif
